In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("Social_Network_Ads.csv")
data.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


## Data Pre - processing

In [3]:
# split data into train set and test set

X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 0] = le.fit_transform(X[:, 0])

from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()
X = min_max.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)

In [14]:
bias = 1
w = np.random.random(X_train.shape[1])
function = X_test@ w.T + bias* np.ones_like(y_test)

array([1.21375964, 1.4832956 , 1.1482582 , 1.42849185, 1.11179336,
       1.2110181 , 1.45138774, 1.26105189, 1.4553495 , 1.44212323,
       1.2539386 , 1.29501591, 1.12433424, 1.03562198, 1.41835674,
       1.51012549, 1.29891525, 1.35487679, 1.35883855, 1.52593582,
       1.0603888 , 1.39240438, 1.36292312, 1.4507301 , 1.15730485,
       1.39074642, 1.4828301 , 1.37477861, 1.55588258, 1.02323857,
       1.26532168, 1.29570821, 1.6147778 , 1.38273476, 1.54219082,
       1.42057037, 1.38020619, 1.27480607, 1.23584733, 1.14383095,
       1.39701686, 1.15028661, 1.23990414, 1.35478174, 1.24567442,
       1.51736159, 1.34084234, 1.24239804, 1.41872719, 1.18727936,
       1.36186729, 1.4252779 , 1.24766816, 1.36458108, 1.30855712,
       1.10602309, 1.66773836, 1.33516712, 1.39003328, 1.39546086,
       1.25768047, 1.23222826, 1.44003241, 1.26495123, 1.32845893,
       1.39368498, 1.37047416, 1.55179802, 1.26547915, 1.37028893,
       1.57029439, 1.33641509, 1.31878932, 1.18784194, 1.39277

## Support Vector Machine - Sklearn

In [5]:
from sklearn.svm import SVC

# using SVM with linear kernel to classify data
classifier = SVC(kernel = "linear", random_state = 42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
comparison_array = np.equal(y_pred, y_test)

print(np.sum(comparison_array) * 1./len(y_pred))

0.875


## Support Vector Machine from scratch

In [22]:
class SVM():
    def __init__(self, learning_rate, num_epochs, Lambda):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.Lambda = Lambda
        self.w = None
        self.bias = None
        
    def fit(self, X_train, y_train):
        np.random.seed(42)
        
        self.w = np.random.random(X_train.shape[1])
        self.bias = 0
        
        # convert y_train set into the set of 2 classes {1, -1}
        for index in range(len(y_train)):
            if y_train[index] == 0:
                y_train[index] = -1
        
        # using gradient descent to minimize cost function
        for epoch in range(self.num_epochs):
            cost = self.Lambda * self.w
            for index, instance in enumerate(X_train):
                cost += max(1 - y_train[index]* (self.w@ instance.T + self.bias), 0)
                if y_train[index]* (self.w@ instance.T + self.bias) >= 1:
                    self.w -= self.learning_rate* (2* self.Lambda* instance)
                else:
                    self.w -= self.learning_rate* (2* self.Lambda* instance - y_train[index]* instance)
                    self.bias += self.learning_rate* y_train[index]
    
    def predict(self, X_test):
        
        # the hyperplane that we have
        function = X_test@ self.w.T + self.bias* np.ones_like(y_test)
        y_pred = np.sign(function)
        return y_pred

In [25]:
# the accuracy

svm = SVM(learning_rate = 0.001, Lambda = 0.01, num_epochs = 1500)
svm.fit(X_train, y_train)
y_predict = svm.predict(X_test)

# convert type of y_pred from float to interger
y_predict = y_predict.astype(int)

for index in range(len(y_predict)):
    if y_predict[index] == -1:
        y_predict[index] = 0
y_predict

array([0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [27]:
comparison = np.equal(y_predict, y_test)
print(np.sum(comparison)* 1./len(comparison))

0.875
